# 🔮 TFT Model Eğitimi - Google Colab

Bu notebook TFT (Temporal Fusion Transformer) modelini JetX verileri üzerinde eğitir.

## 📋 TFT Özellikleri:
- **Multi-horizon**: Çoklu zaman adımı tahmini
- **Interpretable**: Attention mechanism ile yorumlanabilir
- **JetX Enhanced**: JetX-specific feature engineering
- **GPU Accelerated**: PyTorch CUDA desteği

---

**⚠️ Önce `colab_setup.ipynb` notebook'unu çalıştırın!**

In [ ]:
# 🔧 Kurulum ve Import'lar
import sys
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time
from datetime import datetime

# Proje path'i ekle
sys.path.insert(0, '/content/predictor_1/src')

# Model import'ları
from models.deep_learning.tft.tft_model import TFTPredictor
from data_processing.loader import load_data_from_sqlite
from data_processing.splitter import create_sequences

# GPU kontrolü
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🎯 Device: {device}")
if torch.cuda.is_available():
    print(f"📱 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# Matplotlib için stil
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ TFT kurulumu tamamlandı!")

In [ ]:
# 📊 Veri Yükleme
print("📊 JETX VERİSİ YÜKLEME")
print("=" * 50)

# Colab'dan veritabanını yükle
db_path = "/content/colab_jetx_data.db"
df = load_data_from_sqlite(db_path)

# Veriyi liste formatına çevir
jetx_data = [row[1] for row in df.data]  # value column
print(f"✅ {len(jetx_data)} JetX verisi yüklendi")

# Veri analizi
jetx_array = np.array(jetx_data)
print(f"📈 İstatistikler:")
print(f"   - Min: {jetx_array.min():.2f}")
print(f"   - Max: {jetx_array.max():.2f}")
print(f"   - Mean: {jetx_array.mean():.2f}")
print(f"   - Std: {jetx_array.std():.2f}")
print(f"   - Crash rate (<1.5): {(jetx_array < 1.5).mean()*100:.1f}%")

# TFT için veri görselleştirmesi
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Time series plot
axes[0,0].plot(jetx_data[:300])
axes[0,0].set_title('İlk 300 JetX Değeri')
axes[0,0].set_xlabel('Zaman')
axes[0,0].set_ylabel('JetX Değeri')
axes[0,0].axhline(y=1.5, color='r', linestyle='--', alpha=0.7, label='Threshold')
axes[0,0].legend()

# Moving average
ma_10 = [np.mean(jetx_data[max(0, i-10):i+1]) for i in range(len(jetx_data))]
axes[0,1].plot(jetx_data[:300], alpha=0.7, label='Original')
axes[0,1].plot(ma_10[:300], label='MA(10)')
axes[0,1].set_title('JetX ile Moving Average')
axes[0,1].set_xlabel('Zaman')
axes[0,1].set_ylabel('JetX Değeri')
axes[0,1].legend()

# Distribution by ranges
ranges = ['1.0-1.5', '1.5-2.0', '2.0-3.0', '3.0-5.0', '5.0+']
counts = [
    sum(1 for x in jetx_data if 1.0 <= x < 1.5),
    sum(1 for x in jetx_data if 1.5 <= x < 2.0),
    sum(1 for x in jetx_data if 2.0 <= x < 3.0),
    sum(1 for x in jetx_data if 3.0 <= x < 5.0),
    sum(1 for x in jetx_data if x >= 5.0)
]
axes[1,0].bar(ranges, counts)
axes[1,0].set_title('JetX Değer Aralıkları')
axes[1,0].set_ylabel('Frekans')
axes[1,0].tick_params(axis='x', rotation=45)

# Volatility over time
volatility = [np.std(jetx_data[max(0, i-20):i+1]) for i in range(20, len(jetx_data))]
axes[1,1].plot(volatility[:280])
axes[1,1].set_title('Rolling Volatility (20-period)')
axes[1,1].set_xlabel('Zaman')
axes[1,1].set_ylabel('Volatility')

plt.tight_layout()
plt.show()

print("🎯 TFT veri analizi tamamlandı!")

In [ ]:
# 🔧 TFT İçin Veri Hazırlama
print("🔧 TFT İÇİN VERİ HAZIRLAMA")
print("=" * 50)

# TFT parametreleri
SEQUENCE_LENGTH = 150  # TFT için daha uzun sequence
FORECAST_HORIZON = 1   # Tek adım tahmin
TRAIN_SPLIT = 0.8
VALIDATION_SPLIT = 0.1
BATCH_SIZE = 16  # TFT daha fazla memory kullandığı için

# Veriyi train/val/test olarak böl
n_train = int(len(jetx_data) * TRAIN_SPLIT)
n_val = int(len(jetx_data) * VALIDATION_SPLIT)

train_data = jetx_data[:n_train]
val_data = jetx_data[n_train:n_train + n_val]
test_data = jetx_data[n_train + n_val:]

print(f"📊 Veri bölümü:")
print(f"   - Train: {len(train_data)} samples")
print(f"   - Validation: {len(val_data)} samples")
print(f"   - Test: {len(test_data)} samples")

# TFT için enhanced features
def create_enhanced_features(data, window=20):
    """TFT için gelişmiş özellikler oluştur"""
    features = []
    
    for i in range(len(data)):
        # Temel değer
        value = data[i]
        
        # Moving averages
        start_idx = max(0, i - window + 1)
        window_data = data[start_idx:i+1]
        
        ma_5 = np.mean(window_data[-5:]) if len(window_data) >= 5 else value
        ma_10 = np.mean(window_data[-10:]) if len(window_data) >= 10 else value
        
        # Volatility
        volatility = np.std(window_data) if len(window_data) > 1 else 0
        
        # Momentum
        momentum = (value - window_data[0]) / window_data[0] if len(window_data) > 0 and window_data[0] != 0 else 0
        
        # Threshold indicators
        above_threshold = 1 if value >= 1.5 else 0
        crash_indicator = 1 if value < 1.5 else 0
        
        features.append([
            value,
            ma_5,
            ma_10,
            volatility,
            momentum,
            above_threshold,
            crash_indicator
        ])
    
    return np.array(features)

# Enhanced features oluştur
print("🔧 Enhanced features oluşturuluyor...")
train_features = create_enhanced_features(train_data)
val_features = create_enhanced_features(val_data)
test_features = create_enhanced_features(test_data)

print(f"📊 Feature boyutları:")
print(f"   - Train features: {train_features.shape}")
print(f"   - Val features: {val_features.shape}")
print(f"   - Test features: {test_features.shape}")

# Normalizasyon
train_mean = np.mean(train_features, axis=0)
train_std = np.std(train_features, axis=0)
train_std[train_std == 0] = 1  # Divide by zero'yu önle

print(f"📏 Normalizasyon parametreleri:")
print(f"   - Feature means: {train_mean}")
print(f"   - Feature stds: {train_std}")

print("✅ TFT veri hazırlama tamamlandı!")

In [ ]:
# 🔮 TFT Model Oluşturma
print("🔮 TFT MODEL OLUŞTURMA")
print("=" * 50)

# TFT model parametreleri
model_params = {
    'sequence_length': SEQUENCE_LENGTH,
    'hidden_size': 128,  # TFT için optimize edilmiş boyut
    'num_heads': 4,      # Attention head sayısı
    'num_layers': 2,     # Transformer katman sayısı
    'learning_rate': 0.001
}

print(f"🔧 TFT model parametreleri:")
for key, value in model_params.items():
    print(f"   - {key}: {value}")

# TFT model oluştur
tft_model = TFTPredictor(**model_params)

# GPU'ya taşı
if torch.cuda.is_available():
    tft_model.model = tft_model.model.cuda()
    print("✅ TFT model GPU'ya taşındı")

# Model özeti
total_params = sum(p.numel() for p in tft_model.model.parameters())
trainable_params = sum(p.numel() for p in tft_model.model.parameters() if p.requires_grad)

print(f"📊 TFT model istatistikleri:")
print(f"   - Total parameters: {total_params:,}")
print(f"   - Trainable parameters: {trainable_params:,}")
print(f"   - Model size: ~{total_params * 4 / 1024**2:.1f} MB")

print("✅ TFT model hazır!")

In [ ]:
# 🚀 TFT Model Eğitimi
print("🚀 TFT MODEL EĞİTİMİ")
print("=" * 50)

# Eğitim parametreleri
EPOCHS = 40  # TFT için biraz daha az epoch
VERBOSE = True

print(f"🎯 TFT eğitimi başlıyor...")
print(f"   - Epochs: {EPOCHS}")
print(f"   - Batch size: {BATCH_SIZE}")
print(f"   - Sequence length: {SEQUENCE_LENGTH}")
print(f"   - Device: {device}")

# Eğitim başlangıç zamanı
start_time = time.time()

# TFT model eğitimi
try:
    history = tft_model.train(
        data=train_data,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.2,
        verbose=VERBOSE
    )
    
    # Eğitim süresi
    training_time = time.time() - start_time
    print(f"\n⏱️ TFT eğitim süresi: {training_time/60:.2f} dakika")
    
    # Eğitim sonuçları
    final_train_loss = history['train_losses'][-1]
    final_val_loss = history['val_losses'][-1]
    
    print(f"📊 TFT final sonuçlar:")
    print(f"   - Train Loss: {final_train_loss:.6f}")
    print(f"   - Val Loss: {final_val_loss:.6f}")
    print(f"   - Epochs completed: {len(history['train_losses'])}")
    
    print("✅ TFT model eğitimi tamamlandı!")
    
except Exception as e:
    print(f"❌ TFT eğitim hatası: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# 📈 TFT Eğitim Sonuçları Görselleştirme
if 'history' in locals() and history:
    print("📈 TFT EĞİTİM SONUÇLARI GÖRSELLEŞTİRME")
    print("=" * 50)
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss curves
    epochs_range = range(1, len(history['train_losses']) + 1)
    
    axes[0,0].plot(epochs_range, history['train_losses'], 'b-', label='Training Loss', linewidth=2)
    axes[0,0].plot(epochs_range, history['val_losses'], 'r-', label='Validation Loss', linewidth=2)
    axes[0,0].set_title('TFT Model Loss', fontsize=14)
    axes[0,0].set_xlabel('Epochs')
    axes[0,0].set_ylabel('Loss')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # Loss improvement rate
    train_improvements = []
    val_improvements = []
    
    for i in range(1, len(history['train_losses'])):
        train_imp = history['train_losses'][i-1] - history['train_losses'][i]
        val_imp = history['val_losses'][i-1] - history['val_losses'][i]
        train_improvements.append(train_imp)
        val_improvements.append(val_imp)
    
    axes[0,1].plot(range(2, len(epochs_range) + 1), train_improvements, 'g-', label='Train Improvement', alpha=0.7)
    axes[0,1].plot(range(2, len(epochs_range) + 1), val_improvements, 'orange', label='Val Improvement', alpha=0.7)
    axes[0,1].set_title('Loss Improvement Rate')
    axes[0,1].set_xlabel('Epochs')
    axes[0,1].set_ylabel('Loss Improvement')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    axes[0,1].axhline(y=0, color='k', linestyle='--', alpha=0.5)
    
    # Training efficiency
    training_efficiency = [history['train_losses'][i] / history['val_losses'][i] for i in range(len(history['train_losses']))]
    axes[1,0].plot(epochs_range, training_efficiency, 'purple', linewidth=2)
    axes[1,0].set_title('Training Efficiency (Train/Val Loss)')
    axes[1,0].set_xlabel('Epochs')
    axes[1,0].set_ylabel('Efficiency Ratio')
    axes[1,0].grid(True, alpha=0.3)
    axes[1,0].axhline(y=1, color='r', linestyle='--', alpha=0.5, label='Perfect Efficiency')
    axes[1,0].legend()
    
    # Loss statistics table
    loss_stats = {
        'Best Train Loss': f"{min(history['train_losses']):.6f}",
        'Best Val Loss': f"{min(history['val_losses']):.6f}",
        'Final Train Loss': f"{history['train_losses'][-1]:.6f}",
        'Final Val Loss': f"{history['val_losses'][-1]:.6f}",
        'Training Time': f"{training_time/60:.2f} min"
    }
    
    axes[1,1].axis('off')
    table_data = [[k, v] for k, v in loss_stats.items()]
    table = axes[1,1].table(cellText=table_data, colLabels=['Metric', 'Value'], loc='center', cellLoc='left')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.5)
    
    # Tablo stilini güzelleştir
    for i in range(len(table_data) + 1):
        for j in range(2):
            cell = table[(i, j)]
            if i == 0:  # Header
                cell.set_facecolor('#40466e')
                cell.set_text_props(weight='bold', color='white')
            else:
                cell.set_facecolor('#f1f1f2')
    
    plt.suptitle('TFT Model Training Results', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("✅ TFT eğitim sonuçları görselleştirildi!")
else:
    print("⚠️ TFT eğitim history bulunamadı.")

In [ ]:
# 🧪 TFT Model Test ve Değerlendirme
print("🧪 TFT MODEL TEST VE DEĞERLENDİRME")
print("=" * 50)

# Test verisi üzerinde tahminler
test_predictions = []
test_actuals = []

print("🔍 TFT test verisi üzerinde tahminler yapılıyor...")

for i in tqdm(range(len(test_data) - SEQUENCE_LENGTH)):
    # Sequence hazırla
    sequence = test_data[i:i + SEQUENCE_LENGTH]
    actual = test_data[i + SEQUENCE_LENGTH]
    
    try:
        # TFT tahmin yap
        predicted_value = tft_model.predict(sequence)
        
        test_predictions.append(predicted_value)
        test_actuals.append(actual)
        
    except Exception as e:
        print(f"⚠️ TFT tahmin hatası: {e}")
        break

print(f"✅ TFT {len(test_predictions)} tahmin yapıldı")

# TFT metrikleri hesapla
if test_predictions:
    test_predictions = np.array(test_predictions)
    test_actuals = np.array(test_actuals)
    
    # Regression metrikleri
    mae = np.mean(np.abs(test_predictions - test_actuals))
    mse = np.mean((test_predictions - test_actuals) ** 2)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((test_predictions - test_actuals) / test_actuals)) * 100
    
    # R² score
    ss_res = np.sum((test_actuals - test_predictions) ** 2)
    ss_tot = np.sum((test_actuals - np.mean(test_actuals)) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    
    # Classification metrikleri (threshold = 1.5)
    actual_binary = (test_actuals >= 1.5).astype(int)
    pred_binary = (test_predictions >= 1.5).astype(int)
    
    accuracy = np.mean(actual_binary == pred_binary)
    precision = np.sum((pred_binary == 1) & (actual_binary == 1)) / np.sum(pred_binary == 1) if np.sum(pred_binary == 1) > 0 else 0
    recall = np.sum((pred_binary == 1) & (actual_binary == 1)) / np.sum(actual_binary == 1) if np.sum(actual_binary == 1) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    print(f"📊 TFT TEST SONUÇLARI:")
    print(f"   Regression Metrics:")
    print(f"   - MAE: {mae:.4f}")
    print(f"   - RMSE: {rmse:.4f}")
    print(f"   - MAPE: {mape:.2f}%")
    print(f"   - R² Score: {r2:.4f}")
    print(f"   Classification Metrics:")
    print(f"   - Accuracy: {accuracy:.4f}")
    print(f"   - Precision: {precision:.4f}")
    print(f"   - Recall: {recall:.4f}")
    print(f"   - F1-Score: {f1:.4f}")

print("✅ TFT model değerlendirmesi tamamlandı!")

In [ ]:
# 📊 TFT Tahmin Sonuçları Görselleştirme
if 'test_predictions' in locals() and len(test_predictions) > 0:
    print("📊 TFT TAHMİN SONUÇLARI GÖRSELLEŞTİRME")
    print("=" * 50)
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Actual vs Predicted scatter
    axes[0,0].scatter(test_actuals, test_predictions, alpha=0.6, s=30)
    axes[0,0].plot([test_actuals.min(), test_actuals.max()], [test_actuals.min(), test_actuals.max()], 'r--', alpha=0.8, linewidth=2)
    axes[0,0].set_xlabel('Actual Values')
    axes[0,0].set_ylabel('Predicted Values')
    axes[0,0].set_title('TFT: Actual vs Predicted')
    axes[0,0].grid(True, alpha=0.3)
    
    # Time series comparison
    n_show = min(150, len(test_predictions))
    x_axis = range(n_show)
    axes[0,1].plot(x_axis, test_actuals[:n_show], 'b-', label='Actual', alpha=0.8, linewidth=2)
    axes[0,1].plot(x_axis, test_predictions[:n_show], 'r-', label='TFT Predicted', alpha=0.8, linewidth=2)
    axes[0,1].axhline(y=1.5, color='g', linestyle='--', alpha=0.6, label='Threshold', linewidth=2)
    axes[0,1].set_xlabel('Time Steps')
    axes[0,1].set_ylabel('JetX Value')
    axes[0,1].set_title(f'TFT: Time Series Comparison (İlk {n_show} tahmin)')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    
    # Error distribution
    errors = test_predictions - test_actuals
    axes[0,2].hist(errors, bins=40, alpha=0.7, edgecolor='black', color='skyblue')
    axes[0,2].axvline(x=0, color='r', linestyle='--', alpha=0.8, linewidth=2)
    axes[0,2].axvline(x=np.mean(errors), color='orange', linestyle='-', alpha=0.8, linewidth=2, label=f'Mean: {np.mean(errors):.3f}')
    axes[0,2].set_xlabel('Prediction Error')
    axes[0,2].set_ylabel('Frequency')
    axes[0,2].set_title('TFT: Error Distribution')
    axes[0,2].legend()
    axes[0,2].grid(True, alpha=0.3)
    
    # Residuals vs Fitted
    axes[1,0].scatter(test_predictions, errors, alpha=0.6, s=30)
    axes[1,0].axhline(y=0, color='r', linestyle='--', alpha=0.8, linewidth=2)
    axes[1,0].set_xlabel('Predicted Values')
    axes[1,0].set_ylabel('Residuals')
    axes[1,0].set_title('TFT: Residuals vs Fitted')
    axes[1,0].grid(True, alpha=0.3)
    
    # Prediction intervals
    n_intervals = min(50, len(test_predictions))
    x_intervals = range(n_intervals)
    axes[1,1].fill_between(x_intervals, 
                          test_predictions[:n_intervals] - np.std(errors),
                          test_predictions[:n_intervals] + np.std(errors),
                          alpha=0.3, color='gray', label='±1σ Interval')
    axes[1,1].plot(x_intervals, test_actuals[:n_intervals], 'b-', label='Actual', linewidth=2)
    axes[1,1].plot(x_intervals, test_predictions[:n_intervals], 'r-', label='TFT Predicted', linewidth=2)
    axes[1,1].set_xlabel('Time Steps')
    axes[1,1].set_ylabel('JetX Value')
    axes[1,1].set_title('TFT: Prediction Intervals')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
    
    # Performance metrics visualization
    metrics = ['MAE', 'RMSE', 'MAPE', 'R²', 'Accuracy', 'F1-Score']
    values = [mae, rmse, mape/10, r2, accuracy, f1]  # MAPE'yi 10'a böldük görselleştirme için
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7', '#DDA0DD']
    
    bars = axes[1,2].bar(metrics, values, color=colors, alpha=0.8)
    axes[1,2].set_ylabel('Score')
    axes[1,2].set_title('TFT: Performance Metrics')
    axes[1,2].tick_params(axis='x', rotation=45)
    
    # Değerleri barların üstüne yaz
    for bar, value in zip(bars, values):
        height = bar.get_height()
        axes[1,2].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                      f'{value:.3f}', ha='center', va='bottom', fontsize=9)
    
    plt.suptitle('TFT Model Prediction Analysis', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("✅ TFT tahmin sonuçları görselleştirildi!")
else:
    print("⚠️ TFT tahmin sonuçları bulunamadı.")

In [ ]:
# 💾 TFT Model Kaydetme
print("💾 TFT MODEL KAYDETME")
print("=" * 50)

# Model kaydetme dizini
model_dir = "/content/trained_models/"
os.makedirs(model_dir, exist_ok=True)

# Model dosya adı
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_filename = f"tft_model_{timestamp}.pth"
model_path = os.path.join(model_dir, model_filename)

# TFT model kaydet
try:
    tft_model.save_model(model_path)
    print(f"✅ TFT model kaydedildi: {model_path}")
    
    # Model bilgilerini kaydet
    model_info = {
        'model_type': 'TFT (Temporal Fusion Transformer)',
        'timestamp': timestamp,
        'parameters': model_params,
        'training_time': f"{training_time/60:.2f} minutes" if 'training_time' in locals() else 'Unknown',
        'final_losses': {
            'train': history['train_losses'][-1] if 'history' in locals() else 'Unknown',
            'val': history['val_losses'][-1] if 'history' in locals() else 'Unknown'
        },
        'test_metrics': {
            'mae': mae if 'mae' in locals() else 'Unknown',
            'rmse': rmse if 'rmse' in locals() else 'Unknown',
            'mape': mape if 'mape' in locals() else 'Unknown',
            'r2': r2 if 'r2' in locals() else 'Unknown',
            'accuracy': accuracy if 'accuracy' in locals() else 'Unknown',
            'f1_score': f1 if 'f1' in locals() else 'Unknown'
        },
        'data_info': {
            'sequence_length': SEQUENCE_LENGTH,
            'forecast_horizon': FORECAST_HORIZON,
            'train_size': len(train_data),
            'val_size': len(val_data),
            'test_size': len(test_data)
        },
        'model_architecture': {
            'hidden_size': model_params['hidden_size'],
            'num_heads': model_params['num_heads'],
            'num_layers': model_params['num_layers'],
            'total_parameters': total_params if 'total_params' in locals() else 'Unknown'
        }
    }
    
    # Model bilgilerini JSON olarak kaydet
    import json
    info_filename = f"tft_info_{timestamp}.json"
    info_path = os.path.join(model_dir, info_filename)
    
    with open(info_path, 'w') as f:
        json.dump(model_info, f, indent=2)
    
    print(f"✅ TFT model bilgileri kaydedildi: {info_path}")
    
    # Google Drive'a kopyala (optional)
    print("\n💡 Google Drive'a kaydetmek için:")
    print(f"   !cp {model_path} /content/drive/MyDrive/")
    print(f"   !cp {info_path} /content/drive/MyDrive/")
    
    # Model performans özeti
    print("\n📊 TFT MODEL PERFORMANS ÖZETİ:")
    print(f"   🎯 MAE: {mae:.4f}" if 'mae' in locals() else "   🎯 MAE: Unknown")
    print(f"   🎯 RMSE: {rmse:.4f}" if 'rmse' in locals() else "   🎯 RMSE: Unknown")
    print(f"   🎯 R² Score: {r2:.4f}" if 'r2' in locals() else "   🎯 R² Score: Unknown")
    print(f"   🎯 Accuracy: {accuracy:.4f}" if 'accuracy' in locals() else "   🎯 Accuracy: Unknown")
    print(f"   ⏱️ Training Time: {training_time/60:.2f} minutes" if 'training_time' in locals() else "   ⏱️ Training Time: Unknown")
    
except Exception as e:
    print(f"❌ TFT model kaydetme hatası: {e}")
    import traceback
    traceback.print_exc()

print("\n🎉 TFT model eğitimi tamamlandı!")

# 🎯 TFT Eğitimi Özeti

## ✅ Tamamlanan İşlemler:
1. **Veri Hazırlama** - JetX verisi enhanced features ile işlendi
2. **Model Oluşturma** - TFT modeli GPU'da oluşturuldu
3. **Eğitim** - Model attention mechanisms ile eğitildi
4. **Değerlendirme** - Kapsamlı test metrikleri hesaplandı
5. **Görselleştirme** - Detaylı analiz grafikleri oluşturuldu
6. **Kaydetme** - Model ve performans bilgileri kaydedildi

## 📊 TFT Model Özellikleri:
- **Architecture**: Temporal Fusion Transformer
- **Input**: Enhanced JetX features (7 features)
- **Output**: Continuous value prediction
- **Attention**: Multi-head self-attention mechanism
- **Device**: GPU accelerated with CUDA

## 🚀 TFT Avantajları:
- **Interpretability**: Attention weights ile yorumlanabilir
- **Multi-horizon**: Çoklu zaman adımı desteği
- **Feature Integration**: Çoklu özellik entegrasyonu
- **Temporal Patterns**: Uzun dönem bağımlılıkları yakalama

## 📈 Sonraki Adımlar:
1. **LSTM Eğitimi** - Sequence modeling ile karşılaştırma
2. **Ensemble Model** - TFT + N-Beats kombinasyonu
3. **Attention Analysis** - Hangi özelliklerin önemli olduğu analizi
4. **Real-time Deployment** - Canlı tahmin servisi

---

**🎉 TFT eğitimi başarıyla tamamlandı!**